In [46]:
import pandas as pd
import os

# Get the current working directory (where the notebook is located)
folder_path = os.getcwd()

# List to hold all dataframes
dfs = []

# Loop through all files in the current directory
for file in os.listdir(folder_path):
    if file.endswith(".XLSX") and not file.startswith("combined_file"):  # Skip the combined file if it exists
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path)
        dfs.append(df)

# Combine all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined dataframe to a new Excel file
combined_file_path = os.path.join(folder_path, "combined_file.xlsx")
combined_df.to_excel(combined_file_path, index=False)

print(f"Combined file saved as: {combined_file_path}")

Combined file saved as: C:\Python\WPy64-31241\notebooks\bmw_thesis\combined_file.xlsx


In [7]:
import pandas as pd

# Load the CSV file
input_file = "Protocol.csv"  # Replace with your file path
df = pd.read_csv(input_file)

# Get unique IDENTNR values
unique_idents = df['IDENTNR'].unique()

# Split into chunks of 20 IDENTNR values
chunk_size = 8
for i in range(0, len(unique_idents), chunk_size):
    # Get the current chunk of IDENTNR values
    chunk_idents = unique_idents[i:i + chunk_size]
    
    # Filter the DataFrame for the current chunk
    chunk_df = df[df['IDENTNR'].isin(chunk_idents)]
    
    # Save the chunk to a new CSV file
    output_file = f"output_chunk_{i//chunk_size + 1}.csv"  # File name like output_chunk_1.csv, output_chunk_2.csv, etc.
    chunk_df.to_csv(output_file, index=False)
    
    print(f"Saved {output_file} with {len(chunk_df)} rows.")

Saved output_chunk_1.csv with 16995 rows.
Saved output_chunk_2.csv with 15921 rows.
Saved output_chunk_3.csv with 14095 rows.
Saved output_chunk_4.csv with 13929 rows.
Saved output_chunk_5.csv with 13556 rows.
Saved output_chunk_6.csv with 13922 rows.
Saved output_chunk_7.csv with 13734 rows.
Saved output_chunk_8.csv with 13460 rows.
Saved output_chunk_9.csv with 14800 rows.
Saved output_chunk_10.csv with 15191 rows.
Saved output_chunk_11.csv with 15413 rows.
Saved output_chunk_12.csv with 15215 rows.
Saved output_chunk_13.csv with 14694 rows.
Saved output_chunk_14.csv with 16756 rows.
Saved output_chunk_15.csv with 15969 rows.
Saved output_chunk_16.csv with 15789 rows.
Saved output_chunk_17.csv with 15134 rows.
Saved output_chunk_18.csv with 14977 rows.
Saved output_chunk_19.csv with 17108 rows.
Saved output_chunk_20.csv with 15197 rows.
Saved output_chunk_21.csv with 14159 rows.
Saved output_chunk_22.csv with 13089 rows.
Saved output_chunk_23.csv with 13258 rows.
Saved output_chunk_2

In [44]:
import pandas as pd
from pathlib import Path

# Define the steps as functions for modularity and reusability

def load_data(file_path):
    """Load the dataset from a file."""
    try:
        df = pd.read_excel(file_path)  # Assuming Excel file
        print(f"Data loaded successfully from {file_path}!")
        return df
    except Exception as e:
        print(f"Error loading data from {file_path}: {e}")
        return None


def filter_data(df):
    """
    Filter the dataset based on the conditions:
    - Include all records where Grund der Anfrage is "E", "G", or "I".
    - For other records, include only if Antwortstatus is "APPROVED" and Anforderungsstatus is "RECEIVED".
    """
    try:
        condition1 = df['Grund der Anfrage'].isin(['E', 'G', 'I'])
        condition2 = (df['Antwortstatus'] == 'APPROVED') & (df['Anforderungsstatus'] == 'RECEIVED')
        filtered_df = df[condition1 | condition2]
        print("Data filtered successfully!")
        return filtered_df
    except Exception as e:
        print(f"Error filtering data: {e}")
        return None


def summarize_data(df, agreement_columns):
    """
    Summarize the data by Produktnummer.
    - Concatenate Firmenname.
    - Keep the first occurrence of each agreement column.
    - Include Produktkurztext.
    - Exclude Grund der Anfrage and Anforderungsnr. if they are inconsistent.
    """
    try:
        # Define aggregation rules for each column
        agg_rules = {
            'Firmenname': lambda x: ', '.join(x.astype(str)),
            'Produktkurztext': 'first',  # Include Produktkurztext
            'Grund der Anfrage': lambda x: x.iloc[0] if x.nunique() == 1 else 'MULTIPLE',
            'Anforderungsnr.': lambda x: x.iloc[0] if x.nunique() == 1 else 'MULTIPLE'
        }

        # Add agreement columns to the aggregation rules
        for col in agreement_columns:
            agg_rules[col] = 'first'

        # Group by Produktnummer and aggregate fields
        summarized_df = df.groupby('Produktnummer').agg(agg_rules).reset_index()

        print("Data summarized successfully!")
        return summarized_df
    except Exception as e:
        print(f"Error summarizing data: {e}")
        return None


def modify_fields(df, agreement_columns):
    """
    Rename agreement columns to the desired format.
    Add ZEU1_EGIPREF fields based on each agreement column.
    """
    try:
        # Mapping of agreement columns to their new names
        agreement_mapping = {
            "EUAND - Präf. Krit.": "ZEU3_PREF",
            "EUCAM - Präf. Krit.": "ZEU10_PREF",
            "EUCET - Präf. Krit.": "ZEU5_PREF",
            "EUCAR - Präf. Krit.": "ZEU4_PREF",
            "EUCHI - Präf. Krit.": "ZEU8_PREF",
            "EUGB - Präf. Krit.": "ZEU12_PREF",
            "EUKOR - Präf. Krit.": "ZEU2_PREF",
            "EUMED - Präf. Krit.": "ZEU1_PREF",
            "EUMEX - Präf. Krit.": "ZEU7_PREF",
            "EUSAD - Präf. Krit.": "ZEU9_PREF",
            "EUUKR - Präf. Krit.": "ZEU6_PREF",
            "EUASE - Präf. Krit.": "ZEU11_PREF",
            "EUPEM - Präf. Krit.": "ZEU13_PREF"
        }

        # Rename agreement columns
        for old_col, new_col in agreement_mapping.items():
            if old_col in df.columns:
                df = df.rename(columns={old_col: new_col})
            else:
                print(f"Warning: Column '{old_col}' not found in the dataset.")

        print("Fields modified successfully!")
        return df
    except Exception as e:
        print(f"Error modifying fields: {e}")
        return None


def save_output(df, output_file):
    """Save the final output to a CSV file."""
    try:
        df.to_excel(output_file, index=False)
        print(f"Output saved to {output_file}!")
    except Exception as e:
        print(f"Error saving output: {e}")


def main():
    # Define the input file and agreement columns
    input_file = "combined_file.xlsx"  # Replace with your file path
    output_file = "final_output.xlsx"  # Single output file
    agreement_columns = [
        "EUAND - Präf. Krit.", "EUASE - Präf. Krit.", "EUCAM - Präf. Krit.", 
        "EUCAR - Präf. Krit.", "EUCET - Präf. Krit.", "EUCHI - Präf. Krit.", 
        "EUGB - Präf. Krit.", "EUKOR - Präf. Krit.", "EUMED - Präf. Krit.", 
        "EUMEX - Präf. Krit.", "EUSAD - Präf. Krit.", "EUUKR - Präf. Krit.", 
        "EUPEM - Präf. Krit."
    ]

    # Load the data
    df = load_data(input_file)
    if df is None:
        return

    # Filter the data
    df = filter_data(df)
    if df is None:
        return

    # Summarize the data
    df = summarize_data(df, agreement_columns)
    if df is None:
        return

    # Modify fields
    df = modify_fields(df, agreement_columns)
    if df is None:
        return

    # Save the output
    save_output(df, output_file)



In [45]:
# Run the script
if __name__ == "__main__":
    main()

Data loaded successfully from combined_file.xlsx!
Data filtered successfully!
Data summarized successfully!
Fields modified successfully!
Output saved to final_output.xlsx!


In [49]:
import pandas as pd

def sort_database(df):
    """
    Sort the data by:
    - "Datum Präferenzkalkulation" (descending)
    - "Produktnummer" (descending)
    - "Abkommen" (ascending)
    - "Kennzeichen RWPK / IPK" (descending)
    """
    try:
        sorted_df = df.sort_values(
            by=[
                "Datum Präferenzkalkulation", 
                "Produktnummer", 
                "Abkommen", 
                "Kennzeichen RWPK / IPK"
            ],
            ascending=[False, False, True, False]  # D=Descending, A=Ascending in VBA
        )
        print("Data sorted successfully!")
        return sorted_df
    except Exception as e:
        print(f"Error sorting data: {e}")
        return None


def summarization(df):
    """
    Remove duplicates based on "Produktnummer" and "Abkommen".
    Keep the first occurrence (oldest entry if sorted by date descending).
    """
    try:
        summarized_df = df.drop_duplicates(
            subset=["Produktnummer", "Abkommen"], 
            keep="first"  # Keep the first occurrence (most recent date due to sorting)
        )
        print("Duplicates removed successfully!")
        return summarized_df
    except Exception as e:
        print(f"Error removing duplicates: {e}")
        return None


def join_database(original_df, summarized_df):
    """
    Join the summarized data with the original data to retain only non-duplicate rows.
    """
    try:
        # Columns to match for the join
        join_columns = [
            "Produktnummer", 
            "Datum Präferenzkalkulation", 
            "Abkommen", 
            "Kennzeichen RWPK / IPK"
        ]
        
        # Inner join to keep only matching rows (non-duplicates)
        joined_df = pd.merge(
            original_df,
            summarized_df[join_columns],
            on=join_columns,
            how="inner"
        )
        print("Data joined successfully!")
        return joined_df
    except Exception as e:
        print(f"Error joining data: {e}")
        return None


def main():
    # Load data
    input_file = "combined_file.xlsx"  # Replace with your file path
    output_file = "final_output.xlsx"  # Output file path

    try:
        df = pd.read_excel(input_file)
        print("Data loaded successfully!")
    except Exception as e:
        print(f"Error loading data: {e}")
        return

    # Step 1: Sort the data
    sorted_df = sort_database(df)
    if sorted_df is None:
        return

    # Step 2: Remove duplicates
    summarized_df = summarization(sorted_df)
    if summarized_df is None:
        return

    # Step 3: Join to retain non-duplicates
    final_df = join_database(df, summarized_df)
    if final_df is None:
        return

    # Save the final output
    try:
        final_df.to_excel(output_file, index=False)
        print(f"Output saved to {output_file}!")
    except Exception as e:
        print(f"Error saving output: {e}")




In [50]:

if __name__ == "__main__":
    main()

Data loaded successfully!
Data sorted successfully!
Duplicates removed successfully!
Data joined successfully!
Output saved to final_output.xlsx!
